In [ ]:
#!pip install aiohttp

### 0. Prepare session variables

In [2]:
import dsx_core_utils
DSXHI_SYSTEMS = dsx_core_utils.get_dsxhi_info(showSummary=False)

Available Hadoop systems: 

['CDH-513-Stable', 'Chell Edge [shad1]', 'DataPuddle6', 'SL-Prod-HDP', 'ak-cdh632-edge-1', 'asgardian-edge', 'bosom', 'epizoon1', 'outsider513', 'yccdh5']


In [84]:
import os, json
import logging, requests, json, time
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p',level=logging.INFO)

# myConfig={'driverMemory':'4G',"conf":{"livy.rsc.server.connect.timeout":"300s","livy.rsc.launcher.port.range":"10000~101111"}}

# User INPUT
USER_DSXHI_SYSTEM="outsider513"
USER_DSXHI_IMAGEID="jupyter-py36"
import dsx_core_utils
HI_CONFIG = dsx_core_utils.setup_livy_sparkmagic(
                          system=USER_DSXHI_SYSTEM, 
#                           imageId=USER_DSXHI_IMAGEID,
#                           addlConfig=myConfig,
                          livy="livyspark2")
# %reload_ext sparkmagic.magics
# import sparkmagic.utils.configuration as sm_conf
# sm_conf.override(sm_conf.livy_session_startup_timeout_seconds.__name__, 300)


sparkmagic has been configured to use https://ibm-dsxhi-outsider513-svc:8443/livy2/v1 
success configuring sparkmagic livy.


In [104]:
headers = {'Content-Type':"application/json",
           'Authorization':"Bearer {}".format(os.environ['USER_ACCESS_TOKEN']),
           'X-Requested-By': 'dsx'}

import sparkmagic.utils.configuration as sm_conf
LIVY_PAYLOAD = sm_conf.session_configs()
LIVY_PAYLOAD['conf']['livy.rsc.server.connect.timeout']='300s'
LIVY_PAYLOAD['driverCores']=1
LIVY_PAYLOAD['driverMemory']='650M'
LIVY_PAYLOAD['executorMemory']='500M'
LIVY_PAYLOAD['kind']='pyspark'

In [105]:
LIVY_PAYLOAD

{'proxyUser': 'user1',
 'conf': {'spark.yarn.appMasterEnv.HI_UTILS_PATH': '/user/dsxhi/environments/pythonAddons/hi_core_utils.zip',
  'livy.rsc.server.connect.timeout': '300s'},
 'driverMemory': '650M',
 'executorMemory': '500M',
 'kind': 'pyspark',
 'driverCores': 1}

## 1. Define Methods used for testing

In [106]:
def start_livy_session():
    '''
    returns: sessionId if successful
    '''
    livy_url="{}/sessions".format(HI_CONFIG['LIVY'])
    livy_session_req = requests.post(livy_url, 
                               data=json.dumps(LIVY_PAYLOAD),
                               headers=headers, verify=False)
    return livy_session_req

def check_livy_sessions(session_id):
    '''
    Checks whether a livy session is started
    '''
    livy_url="{}/sessions/{}".format(HI_CONFIG['LIVY'],session_id)
    livy_session_req = requests.get(livy_url, 
                               headers=headers, verify=False)
    
    if livy_session_req:
        if livy_session_req.json()['state']=='idle':
            logging.info("Session %s started successfully",session_id)
        if livy_session_req.json()['state']=='starting':
            logging.info("Session %s still starting",session_id)
        if livy_session_req.json()['state']=='dead':
            logging.info("Session %s failed to start",session_id)
    else:
        logging.error("Session Not found: %s",session_id)
        return False
    return livy_session_req

def post_livy_statement(session_id):
    '''
    -X POST \
        ${dsxhi_svc_url}/livy2/v1/sessions/${USER2_SESSION_ID}/statements -d'{"code":"d=2"}'
    '''
    livy_url="{}/sessions/{}/statements".format(HI_CONFIG['LIVY'],session_id)
    stmt_payload={"code":"d=2\nprint('Ran successfull on yarn:{}'.format(sc.version))"}
    livy_stmt_req = requests.post(livy_url, 
                               data=json.dumps(stmt_payload),
                               headers=headers, verify=False)
    return livy_stmt_req

def check_livy_statement(session_id):
    livy_url="{}/sessions/{}/statements/0".format(HI_CONFIG['LIVY'],session_id)
    livy_stmt_req = requests.get(livy_url, 
                               headers=headers, verify=False)
    return livy_stmt_req

def delete_livy_session(session_id):
    '''
    Deletes a single livy session
    '''
    livy_url="{}/sessions".format(HI_CONFIG['LIVY'])
    delete_session_url="{}/{}".format(livy_url,session_id)
    
    delete_req = requests.delete(delete_session_url, 
                                headers=headers, verify=False)
    return delete_req

In [39]:
# new_session=start_livy_session()
# new_session.json()
# del_session = delete_livy_session(1090)
# del_session.status_code
# del_session.json()

In [40]:
# check_livy_sessions(1090)

03/18/2020 08:09:54 PM Session Not found: 1090


False

---
# 2. Test Multi-Livy Sessions
- Start sessions
- Wait for sessions to start
- Post a command
- Verify Heap usage on DSXHI Edge Node

  ```
  0. Grab the Livy PID and Knox PID, for each:

  1. Run /usr/java/jdk1.8.0_172/bin/jmap -heap <pid> 
  2. Add up the total "used: " values displayed(Eden,From,To,PS) 
  3. Check CPU usage of Livy PID / Knox PID (top -i)
  ```
  
- Delete Sessions

### 2.1 - Test 5 sessions

- **Enable Yarn DRF**

In [107]:
# Start sessions
list_of_session_ids=[]
sesh_count = 0
for x in range(0,50):
    new_session=start_livy_session()
    print ("Session: {}, Response: {}".format(sesh_count, new_session))
    if new_session and new_session.json()['id']:
        list_of_session_ids.append(new_session.json()['id'])
        sesh_count += 1
    time.sleep(3)
print(list_of_session_ids)
print("Number of sessions: {}".format(sesh_count))

Session: 0, Response: <Response [201]>
Session: 1, Response: <Response [201]>
Session: 2, Response: <Response [201]>
Session: 3, Response: <Response [201]>
Session: 4, Response: <Response [201]>
Session: 5, Response: <Response [201]>
Session: 6, Response: <Response [201]>
Session: 7, Response: <Response [201]>
Session: 8, Response: <Response [201]>
Session: 9, Response: <Response [201]>
Session: 10, Response: <Response [201]>
Session: 11, Response: <Response [201]>
Session: 12, Response: <Response [201]>
Session: 13, Response: <Response [201]>
Session: 14, Response: <Response [201]>
Session: 15, Response: <Response [201]>
Session: 16, Response: <Response [201]>
Session: 17, Response: <Response [201]>
Session: 18, Response: <Response [201]>
Session: 19, Response: <Response [201]>
Session: 20, Response: <Response [201]>
Session: 21, Response: <Response [201]>
Session: 22, Response: <Response [201]>
Session: 23, Response: <Response [201]>
Session: 24, Response: <Response [201]>
Session: 2

In [110]:
#Wait for sessions to start
for x in list_of_session_ids:
    check_livy_sessions(x)

03/20/2020 02:35:06 AM Session 1685 started successfully
03/20/2020 02:35:06 AM Session 1686 started successfully
03/20/2020 02:35:06 AM Session 1687 started successfully
03/20/2020 02:35:06 AM Session 1688 started successfully
03/20/2020 02:35:06 AM Session 1689 started successfully
03/20/2020 02:35:06 AM Session 1690 started successfully
03/20/2020 02:35:06 AM Session 1691 started successfully
03/20/2020 02:35:06 AM Session 1692 started successfully
03/20/2020 02:35:07 AM Session 1693 started successfully
03/20/2020 02:35:07 AM Session 1694 started successfully
03/20/2020 02:35:07 AM Session 1695 started successfully
03/20/2020 02:35:07 AM Session 1696 started successfully
03/20/2020 02:35:07 AM Session 1697 started successfully
03/20/2020 02:35:07 AM Session 1698 started successfully
03/20/2020 02:35:07 AM Session 1699 started successfully
03/20/2020 02:35:07 AM Session 1700 started successfully
03/20/2020 02:35:07 AM Session 1701 started successfully
03/20/2020 02:35:07 AM Session 

In [58]:
#Post a statement to each session (once all are started)
for x in list_of_session_ids:
    post_livy_statement(x)

In [59]:
#Verify stmt ran
for x in list_of_session_ids:
    print(check_livy_statement(x).json()['output'])

{'status': 'ok', 'execution_count': 0, 'data': {'text/plain': 'Ran successfull on yarn:2.3.0.cloudera3'}}
{'status': 'ok', 'execution_count': 0, 'data': {'text/plain': 'Ran successfull on yarn:2.3.0.cloudera3'}}
{'status': 'ok', 'execution_count': 0, 'data': {'text/plain': 'Ran successfull on yarn:2.3.0.cloudera3'}}
{'status': 'ok', 'execution_count': 0, 'data': {'text/plain': 'Ran successfull on yarn:2.3.0.cloudera3'}}
{'status': 'ok', 'execution_count': 0, 'data': {'text/plain': 'Ran successfull on yarn:2.3.0.cloudera3'}}
{'status': 'ok', 'execution_count': 0, 'data': {'text/plain': 'Ran successfull on yarn:2.3.0.cloudera3'}}
{'status': 'ok', 'execution_count': 0, 'data': {'text/plain': 'Ran successfull on yarn:2.3.0.cloudera3'}}
{'status': 'ok', 'execution_count': 0, 'data': {'text/plain': 'Ran successfull on yarn:2.3.0.cloudera3'}}
{'status': 'ok', 'execution_count': 0, 'data': {'text/plain': 'Ran successfull on yarn:2.3.0.cloudera3'}}
{'status': 'ok', 'execution_count': 0, 'data':

- **PAUSE HERE - CHECK DSXHI EDGE NODE JAVA PROCESSES**

In [111]:
# Delete sessions
for x in list_of_session_ids:
    delete_livy_session(x)
    time.sleep(2)

In [65]:
# Ensure they are deleted
for x in list_of_session_ids:
    check_livy_sessions(x)

03/19/2020 09:35:10 PM Session Not found: 1541
03/19/2020 09:35:10 PM Session Not found: 1542
03/19/2020 09:35:10 PM Session Not found: 1543
03/19/2020 09:35:10 PM Session Not found: 1544
03/19/2020 09:35:10 PM Session Not found: 1545
03/19/2020 09:35:10 PM Session Not found: 1546
03/19/2020 09:35:11 PM Session Not found: 1547
03/19/2020 09:35:11 PM Session Not found: 1548
03/19/2020 09:35:11 PM Session Not found: 1549
03/19/2020 09:35:11 PM Session Not found: 1550
03/19/2020 09:35:11 PM Session Not found: 1551
03/19/2020 09:35:11 PM Session Not found: 1552
03/19/2020 09:35:11 PM Session Not found: 1553
03/19/2020 09:35:11 PM Session Not found: 1554
03/19/2020 09:35:11 PM Session Not found: 1555
03/19/2020 09:35:11 PM Session Not found: 1556
03/19/2020 09:35:11 PM Session Not found: 1557
03/19/2020 09:35:11 PM Session Not found: 1558
03/19/2020 09:35:11 PM Session Not found: 1559
03/19/2020 09:35:11 PM Session Not found: 1560
03/19/2020 09:35:11 PM Session Not found: 1561
03/19/2020 09

---

In [ ]:
check_livy_sessions(594).json()

In [ ]:
async with aiohttp.ClientSession() as session:
    async with session.post(url, json={'test': 'object'})

In [ ]:
import aiohttp
import asyncio

async def fetch(session, url):
    async with session.get(url) as response:
        return await response.text()

async def main():
    async with aiohttp.ClientSession() as session:
        html = await fetch(session, 'http://python.org')
        print(html)

if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())
